## 0) Cài đặt & Tương thích Kaggle

In [1]:
!pip -q install "pyarrow<17,>=10" "datasets==2.20.0" "transformers==4.44.2"                 "evaluate==0.4.2" "accelerate>=0.31.0" "sentencepiece>=0.1.99"                 "rouge-score==0.1.2" "nltk>=3.8.1"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 68.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 41.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 92.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 11.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━

## 1) Import & Thiết lập

In [2]:
import os
import random
import numpy as np
import nltk
nltk.download("punkt")

import evaluate
from datasets import load_dataset

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
device = "cuda" if torch.cuda.is_available() else "cpu"
device


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-10-04 08:57:42.359029: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759568262.548193      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759568262.608666      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Torch: 2.6.0+cu124
CUDA available: True


'cuda'

## 2) Tải dữ liệu SAMSum

In [3]:
dataset = load_dataset("knkarthick/samsum")
dataset


Generating train split:   0%|          | 0/14731 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14731
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
})

## 3) Cấu hình Tokenizer/Model

In [4]:
model_name = "google/pegasus-large"  # hoặc "google/pegasus-samsum"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to(device)

max_input_length  = 512
max_target_length = 128


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

## 4) Tiền xử lý

In [5]:
def preprocess_function(batch):
    inputs = batch["dialogue"]
    targets = batch["summary"]
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=max_target_length,
            truncation=True
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)
processed_datasets


Map:   0%|          | 0/14731 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14731
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
})

## 5) Data Collator & ROUGE metric

In [6]:
from rouge_score import rouge_scorer, scoring
import numpy as np

# Tạo scorer local (không cần mạng, không bị conflict tên thư mục)
_scorer = rouge_scorer.RougeScorer(
    ['rouge1', 'rouge2', 'rougeL', 'rougeLsum'],
    use_stemmer=True
)

def postprocess_text(preds, labels):
    preds = [p.strip() for p in preds]
    labels = [l.strip() for l in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    # Decode
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Tính ROUGE bằng bootstrap aggregator
    aggregator = scoring.BootstrapAggregator()
    for pred, ref in zip(decoded_preds, decoded_labels):
        aggregator.add_scores(_scorer.score(ref, pred))
    result = aggregator.aggregate()

    # Lấy F1 (mid.fmeasure) và chuyển về %
    out = {
        "rouge1": round(result["rouge1"].mid.fmeasure * 100, 2),
        "rouge2": round(result["rouge2"].mid.fmeasure * 100, 2),
        "rougeL": round(result["rougeL"].mid.fmeasure * 100, 2),
        "rougeLsum": round(result["rougeLsum"].mid.fmeasure * 100, 2),
    }

    # Độ dài trung bình của chuỗi sinh ra
    prediction_lens = [np.count_nonzero(p != tokenizer.pad_token_id) for p in preds]
    out["gen_len"] = float(np.mean(prediction_lens))
    return out


## 6) Thiết lập tham số huấn luyện

In [12]:
output_dir = "pegasus-samsum-finetuned"

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="steps",
    eval_steps=1000,
    save_steps=1000,
    logging_steps=200,
    save_total_limit=2,
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=2,
    predict_with_generate=True,
    generation_max_length=max_target_length,
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=2,
    report_to="none",
    metric_for_best_model="rougeLsum",
    load_best_model_at_end=True
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## 7) Huấn luyện

In [13]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=processed_datasets["train"],
    eval_dataset=processed_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model),
    compute_metrics=compute_metrics,
)

train_result = trainer.train()
train_result


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1000,1.488700,1.406571,50.190000,26.160000,41.860000,41.790000,20.007335
2000,1.388400,1.377837,51.530000,27.590000,43.080000,43.070000,20.169927
3000,1.337400,1.366906,51.610000,27.710000,43.410000,43.370000,20.086797


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'num_bea

TrainOutput(global_step=3682, training_loss=1.451196497513638, metrics={'train_runtime': 6236.8076, 'train_samples_per_second': 4.724, 'train_steps_per_second': 0.59, 'total_flos': 1.1014711712268288e+16, 'train_loss': 1.451196497513638, 'epoch': 1.9995926956757857})

## 8) Đánh giá

In [14]:
val_metrics = trainer.evaluate(eval_dataset=processed_datasets["validation"])
print("Validation metrics:", val_metrics)

test_metrics = trainer.evaluate(eval_dataset=processed_datasets["test"])
print("Test metrics:", test_metrics)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Validation metrics: {'eval_loss': 1.3669055700302124, 'eval_rouge1': 51.58, 'eval_rouge2': 27.67, 'eval_rougeL': 43.43, 'eval_rougeLsum': 43.42, 'eval_gen_len': 20.086797066014668, 'eval_runtime': 447.1285, 'eval_samples_per_second': 1.829, 'eval_steps_per_second': 0.915, 'epoch': 1.9995926956757857}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Test metrics: {'eval_loss': 1.3822301626205444, 'eval_rouge1': 50.85, 'eval_rouge2': 26.89, 'eval_rougeL': 42.85, 'eval_rougeLsum': 42.89, 'eval_gen_len': 19.835164835164836, 'eval_runtime': 458.5388, 'eval_samples_per_second': 1.786, 'eval_steps_per_second': 0.894, 'epoch': 1.9995926956757857}


## 9) Sinh tóm tắt mẫu

In [17]:
def generate_summary(dialogue, max_new_tokens=128, num_beams=4):
    inputs = tokenizer(
        dialogue,
        return_tensors="pt",
        truncation=True,
        max_length=max_input_length
    ).to(device)
    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=num_beams,
            length_penalty=1.0,
            early_stopping=True
        )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

sample_dialogue = dataset["test"][0]["dialogue"]
sample_ref      = dataset["test"][0]["summary"]

print("DIALOGUE:\n", sample_dialogue[:800], "...\n")
print("REFERENCE SUMMARY:\n", sample_ref, "\n")

pred = generate_summary(sample_dialogue)
print("MODEL SUMMARY:\n", pred)



DIALOGUE:
 Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye ...

REFERENCE SUMMARY:
 Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry. 

MODEL SUMMARY:
 Amanda can't find Betty's number. Hannah wants Amanda to text Larry.


## 10) Lưu model & tokenizer

In [18]:
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
print("Saved to:", output_dir)


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


Saved to: pegasus-samsum-finetuned


## 11) Test

In [24]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# 🔸 Chọn checkpoint bạn muốn test
model_dir = "/kaggle/working/pegasus-samsum-finetuned/checkpoint-3682"

# 🔹 Load model + tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir).to("cuda" if torch.cuda.is_available() else "cpu")

# 🔹 Hàm sinh tóm tắt
def summarize_text(text, max_input_length=512, max_new_tokens=128, num_beams=4):
    inputs = tokenizer(
        text,
        truncation=True,
        max_length=max_input_length,
        return_tensors="pt"
    ).to(model.device)
    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=num_beams,
            length_penalty=1.0,
            early_stopping=True
        )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 🔹 Ví dụ: đoạn văn (không phải hội thoại)
text = """
Artificial intelligence (AI) has become a transformative force in the 21st century. 
It powers applications ranging from language translation and image recognition to autonomous vehicles and healthcare diagnostics. 
However, the widespread adoption of AI also raises ethical concerns, including bias in algorithms, data privacy, and job displacement. 
Governments and organizations are now working to establish regulations that ensure the safe and fair use of AI technologies.
"""

summary = summarize_text(text)
print("📄 Original Text:\n", text)
print("\n📝 Model Summary:\n", summary)


📄 Original Text:
 
Artificial intelligence (AI) has become a transformative force in the 21st century. 
It powers applications ranging from language translation and image recognition to autonomous vehicles and healthcare diagnostics. 
However, the widespread adoption of AI also raises ethical concerns, including bias in algorithms, data privacy, and job displacement. 
Governments and organizations are now working to establish regulations that ensure the safe and fair use of AI technologies.


📝 Model Summary:
 Artificial intelligence (AI) has become a transformative force in the 21st century. Governments and organizations are working to establish regulations that ensure the safe and fair use of AI technologies.


In [25]:
# ===== 0) Setup & load model =====
import os, re, numpy as np, torch
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rouge_score import rouge_scorer, scoring

# >>> chọn checkpoint để test:
model_dir = "/kaggle/working/pegasus-samsum-finetuned"  # hoặc checkpoint-3682
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir).to(device)

# ===== 1) Utils =====
def summarize_text(text, max_input_length=512, max_new_tokens=96, num_beams=6,
                   min_new_tokens=24, no_repeat_ngram_size=3, sampling=False):
    enc = tokenizer(text, truncation=True, max_length=max_input_length, return_tensors="pt").to(device)
    gen_kwargs = dict(max_new_tokens=max_new_tokens, min_new_tokens=min_new_tokens,
                      no_repeat_ngram_size=no_repeat_ngram_size)
    if sampling:
        gen_kwargs.update(dict(do_sample=True, temperature=0.9, top_p=0.9))
    else:
        gen_kwargs.update(dict(num_beams=num_beams, length_penalty=1.05, early_stopping=True))
    with torch.no_grad():
        out = model.generate(**enc, **gen_kwargs)
    return tokenizer.decode(out[0], skip_special_tokens=True)

def tokens(s): 
    return re.findall(r"\w+", s.lower(), flags=re.UNICODE)

def bigrams(ts): 
    return list(zip(ts, ts[1:]))

def quick_metrics(src, pred, ref=None):
    src_toks, sum_toks = tokens(src), tokens(pred)
    compression = len(sum_toks) / max(1, len(src_toks))
    novelty = sum(1 for t in set(sum_toks) if t not in set(src_toks)) / max(1, len(set(sum_toks)))
    bg = bigrams(sum_toks)
    redundancy = 1 - (len(set(bg)) / max(1, len(bg)))
    scores = {"compression": round(compression, 3),
              "novelty": round(novelty, 3),
              "redundancy": round(redundancy, 3)}
    if ref is not None and len(ref.strip()) > 0:
        scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL','rougeLsum'], use_stemmer=True)
        agg = scoring.BootstrapAggregator()
        agg.add_scores(scorer.score(ref, pred))
        r = agg.aggregate()
        scores.update({
            "rouge1": round(r["rouge1"].mid.fmeasure*100,2),
            "rouge2": round(r["rouge2"].mid.fmeasure*100,2),
            "rougeL": round(r["rougeL"].mid.fmeasure*100,2),
            "rougeLsum": round(r["rougeLsum"].mid.fmeasure*100,2),
        })
    return scores

def print_case(title, src, pred, met):
    print(f"\n=== {title} ===")
    print("SOURCE:", (src if len(src)<=400 else src[:400]+" ..."))
    print("\nSUMMARY:", pred)
    print("\nMETRICS:", met)

# ===== 2) Test set =====
tests = []

# 2.1 Dialogue (in-domain) - casual
tests.append(dict(
    title="Dialogue - Casual (in-domain)",
    src="""A: Hey, did you book the venue for Friday?
B: Not yet, I'm comparing prices. 
A: We need it by tonight. 
B: Okay, I'll finalize the cheaper one and send you the receipt.""",
    ref="They discuss booking a venue; B will choose the cheaper option and send the receipt."
))

# 2.2 Dialogue - multi-speaker with emoji/noise
tests.append(dict(
    title="Dialogue - Multi-speaker + emoji/noise",
    src="""Ann: omg traffic today was insane 😂
Yi: told u to take the subway!!
Ann: next time 😭 btw 7pm dinner?
Ben: I'm in. where? 
Yi: same place as last week. """,
    ref=None
))

# 2.3 Expository paragraph (general)
tests.append(dict(
    title="Expository - General paragraph",
    src="""Artificial intelligence (AI) has become a transformative force in the 21st century. 
It powers applications from translation and vision to autonomous vehicles and healthcare. 
Yet widespread adoption raises concerns about bias, privacy, and job displacement. 
Regulators are crafting policies to ensure safe and fair AI.""",
    ref="AI is reshaping multiple sectors but raises bias, privacy, and job concerns; regulators are creating safeguards."
))

# 2.4 Expository - Technical explainer (longer)
tests.append(dict(
    title="Expository - Technical explainer (long)",
    src=("""Transformers rely on self-attention to weight tokens in a sequence. 
Positional encodings inject order information into token representations. 
During generation, beam search explores multiple candidates to balance quality and diversity. 
Despite strong results, long-context handling and hallucination remain open challenges.""")*3,  # replicate to exceed length
    ref=None
))

# 2.5 News-like report
tests.append(dict(
    title="News - Short report",
    src="""The city council approved a $5 million grant to expand bike lanes across downtown. 
Officials expect construction to begin in March and finish by late summer, aiming to reduce congestion and emissions.""",
    ref="City approves $5M to expand downtown bike lanes, with construction from March to late summer to cut congestion and emissions."
))

# 2.6 Email / Memo
tests.append(dict(
    title="Email - Work memo",
    src="""Hi team,
Please review the Q3 draft by Thursday. Focus on the revenue projections and the assumptions for marketing spend.
We'll meet Friday at 10 AM to finalize. Thanks!""",
    ref="Team must review Q3 draft by Thursday, focusing on revenue and marketing assumptions; final meeting Friday 10 AM."
))

# 2.7 Meeting Notes / Bullet list
tests.append(dict(
    title="Meeting Notes - Bullets",
    src="""Agenda: 
- KPI review (conversion + retention)
- Mobile release v1.2 next Tuesday
- Assign owners for onboarding revamp
Action items:
- Alice: finalize copy
- Bob: QA mobile build
- Chen: dashboard metrics""",
    ref=None
))

# 2.8 Noisy social text / code-mixed
tests.append(dict(
    title="Noisy - Code-mixed + teencode",
    src="""Hnay deadline dí quá nên t ko kịp push 😅. I'll ping u tầm 9pm, merge nhanh nhé. Mai demo vs sếp.""",
    ref="Author will ping at 9pm to merge quickly; demo is tomorrow; deadline pressure today."
))

# 2.9 Very long input (forces truncation)
tests.append(dict(
    title="Very Long - Truncation check",
    src=("""Chapter 1. Introduction... """ * 600),  # artificial long text
    ref=None
))

# 2.10 Vietnamese expository
tests.append(dict(
    title="Vietnamese - Expository",
    src="""Công nghệ chuỗi khối cho phép lưu trữ dữ liệu theo dạng khối nối tiếp, khó bị sửa đổi. 
Ngoài tiền mã hoá, blockchain còn được ứng dụng trong logistics, nhận dạng, và bỏ phiếu điện tử. 
Tuy nhiên, các vấn đề về mở rộng, phí giao dịch và tiêu thụ năng lượng vẫn là rào cản lớn.""",
    ref="Blockchain lưu trữ dữ liệu khó chỉnh sửa và có ứng dụng rộng, nhưng vướng mở rộng, phí và năng lượng."
))

# 2.11 Vietnamese dialogue
tests.append(dict(
    title="Vietnamese - Dialogue (in-domain-ish)",
    src="""A: Tối nay họp nhóm mấy giờ?
B: 8 giờ nha. 
A: Ok, t nhớ share slide trước nhé.
B: Rồi, t gửi trước 7 rưỡi.""",
    ref="Họp nhóm lúc 8h; B sẽ gửi slide trước 7:30."
))

# ===== 3) Run all tests =====
results = []
for t in tests:
    # Với văn bản ngoài miền, thử tăng tính trừu tượng nhẹ
    sampling = ("Dialogue" not in t["title"])
    pred = summarize_text(t["src"], sampling=sampling)
    met = quick_metrics(t["src"], pred, t.get("ref"))
    results.append((t["title"], pred, met))

    print_case(t["title"], t["src"], pred, met)

# Nếu muốn xem bảng gọn:
import pandas as pd
df = pd.DataFrame([
    dict(title=title, **met) for (title, pred, met) in results
])
df



=== Dialogue - Casual (in-domain) ===
SOURCE: A: Hey, did you book the venue for Friday?
B: Not yet, I'm comparing prices. 
A: We need it by tonight. 
B: Okay, I'll finalize the cheaper one and send you the receipt.

SUMMARY: B is comparing prices for the venue for Friday. He will finalize the cheaper one and send the receipt to A by tonight.

METRICS: {'compression': 0.657, 'novelty': 0.2, 'redundancy': 0.0, 'rouge1': 52.63, 'rouge2': 22.22, 'rougeL': 42.11, 'rougeLsum': 42.11}

=== Dialogue - Multi-speaker + emoji/noise ===
SOURCE: Ann: omg traffic today was insane 😂
Yi: told u to take the subway!!
Ann: next time 😭 btw 7pm dinner?
Ben: I'm in. where? 
Yi: same place as last week. 

SUMMARY: Yi told Ann to take the subway. Ben is going to the same place as last week for dinner at 7 pm.

METRICS: {'compression': 0.733, 'novelty': 0.3, 'redundancy': 0.0}

=== Expository - General paragraph ===
SOURCE: Artificial intelligence (AI) has become a transformative force in the 21st century. 


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,title,compression,novelty,redundancy,rouge1,rouge2,rougeL,rougeLsum
0,Dialogue - Casual (in-domain),0.657,0.200,0.0,52.63,22.22,42.11,42.11
1,Dialogue - Multi-speaker + emoji/noise,0.733,0.300,0.0,NaN,NaN,NaN,NaN
2,Expository - General paragraph,0.489,0.000,0.0,21.05,5.56,15.79,15.79
3,Expository - Technical explainer (long),0.175,0.000,0.0,NaN,NaN,NaN,NaN
4,News - Short report,1.000,0.000,0.0,61.54,20.00,53.85,53.85
5,Email - Work memo,0.767,0.048,0.0,58.54,15.38,34.15,34.15
6,Meeting Notes - Bullets,0.679,0.471,0.0,NaN,NaN,NaN,NaN
7,Noisy - Code-mixed + teencode,1.000,0.273,0.0,33.33,5.88,27.78,27.78
8,Very Long - Truncation check,0.016,0.867,0.0,NaN,NaN,NaN,NaN
9,Vietnamese - Expository,0.224,0.769,0.0,13.04,4.55,8.70,8.70
